In [1]:
# Google Drive マウント
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/卒業研究/DeadReckoning_ExperimentalData/ML
%ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/卒業研究/DeadReckoning_ExperimentalData/ML
data/                logs/           model/        Untitled0.ipynb
DeadReckoning.ipynb  makedata.ipynb  others/
graph/               ML_angle.ipynb  __pycache__/
loaddata.ipynb       ML.ipynb        scripts/


In [2]:
import datetime
from typing import List, Tuple

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from tqdm import tqdm

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# plt.style.use('default')
# plt.style.use('dark_background')
plt.style.use('ggplot')

import sys
sys.path.append('/content/drive/My Drive/卒業研究'
                '/DeadReckoning_ExperimentalData/ML/scripts')
import loaddata as ld

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# ML用フォーマットのデータ生成
def makedata(df_rtk, df_acc):
    output_data = {}
    output_data['datetime'], output_data['speed[m/s]'] = [], []
    output_data['accX[m/s^2]'], output_data['accY[m/s^2]'] = [], []
    output_data['accZ[m/s^2]'] = []
    
    for t in df_rtk.asfreq(freq='1S', method='ffill').index:
        df_rtk_part = df_rtk[:t]
        df_acc_part = df_acc[:t]
        df_acc_part = df_acc_part[-50:]
        
        if df_acc_part.shape[0] != 50: continue
        if np.isnan(df_rtk_part['speed_1Hz[m/s]'][-1]): continue
        
        output_data['datetime'].append(t)
        output_data['speed[m/s]'].append(df_rtk_part['speed_1Hz[m/s]'][-1])
        output_data['accX[m/s^2]'].append(df_acc_part['accX[m/s^2]'])
        output_data['accY[m/s^2]'].append(df_acc_part['accY[m/s^2]'])
        output_data['accZ[m/s^2]'].append(df_acc_part['accZ[m/s^2]'])

    return output_data

In [0]:
# ファイル書き込み
def writedata(filename, output_data):
    datanum = 50
    f = open(filename, mode='w')
    text = ''

    # インデックス
    text += 'time[s],speed[m/s],'
    for i in range(datanum): text += 'acc_x({})[G],'.format(i)
    for i in range(datanum): text += 'acc_y({})[G],'.format(i)
    for i in range(datanum): text += 'acc_z({})[G],'.format(i)
    text += '\n'

    # データ
    for i in range(len(output_data['datetime'])):
        text += str(i) + ', '
        text += str(output_data['speed[m/s]'][i]) + ', '
        for j in range(datanum):
            text += str(output_data['accX[m/s^2]'][i][j] / 9.8) + ', '
        for j in range(datanum):
            text += str(output_data['accY[m/s^2]'][i][j] / 9.8) + ', '
        for j in range(datanum):
            text += str(output_data['accZ[m/s^2]'][i][j] / 9.8) + ', '
        text += '\n'

    f.write(text)
    f.close

In [0]:
# ML用フォーマットのデータ生成
def makedata_angle(df_rtk, df_acc):
    output_data = {}
    output_data['datetime'], output_data['omega[rad/s]'] = [], []
    output_data['gyroX[rad/s]'], output_data['gyroY[rad/s]'] = [], []
    output_data['gyroZ[rad/s]'] = []
    output_data['angleX[rad/s]'], output_data['angleY[rad/s]'] = [], []
    output_data['angleZ[rad/s]'] = []
    output_data['accX[m/s^2]'], output_data['accY[m/s^2]'] = [], []
    output_data['accZ[m/s^2]'] = []
    
    for t in df_rtk.asfreq(freq='1S', method='ffill').index:
        df_rtk_part = df_rtk[:t]
        df_acc_part = df_acc[:t]
        df_acc_part = df_acc_part[-150:]
        
        if df_acc_part.shape[0] != 150: continue
        if np.isnan(df_rtk_part['omega_1Hz[rad/s]'][-1]): continue
        
        output_data['datetime'].append(t)
        output_data['omega[rad/s]'].append(df_rtk_part['omega_1Hz[rad/s]'][-1])
        output_data['gyroX[rad/s]'].append(df_acc_part['gyroX[rad/s]'])
        output_data['gyroY[rad/s]'].append(df_acc_part['gyroY[rad/s]'])
        output_data['gyroZ[rad/s]'].append(df_acc_part['gyroZ[rad/s]'])
        output_data['angleX[rad/s]'].append(df_acc_part['angleX[rad/s]'])
        output_data['angleY[rad/s]'].append(df_acc_part['angleY[rad/s]'])
        output_data['angleZ[rad/s]'].append(df_acc_part['angleZ[rad/s]'])
        output_data['accX[m/s^2]'].append(df_acc_part['accX[m/s^2]'])
        output_data['accY[m/s^2]'].append(df_acc_part['accY[m/s^2]'])
        output_data['accZ[m/s^2]'].append(df_acc_part['accZ[m/s^2]'])

    return output_data

In [0]:
# ファイル書き込み
def writedata_angle(filename, output_data):
    datanum = 50 * 3
    f = open(filename, mode='w')
    text = ''

    # インデックス
    text += 'time[s],omega[rad/s],'
    for i in range(datanum): text += 'gyro_x({})[G],'.format(i)
    for i in range(datanum): text += 'gyro_y({})[G],'.format(i)
    for i in range(datanum): text += 'gyro_z({})[G],'.format(i)
    for i in range(datanum): text += 'mag_x({})[G],'.format(i)
    for i in range(datanum): text += 'mag_y({})[G],'.format(i)
    for i in range(datanum): text += 'mag_z({})[G],'.format(i)
    text += '\n'

    # データ
    for i in range(len(output_data['datetime'])):
        text += str(i) + ', '
        text += str(output_data['omega[rad/s]'][i]) + ', '
        for j in range(datanum):
            text += str(output_data['gyroX[rad/s]'][i][j]) + ', '
        for j in range(datanum):
            text += str(output_data['gyroY[rad/s]'][i][j]) + ', '
        for j in range(datanum):
            text += str(output_data['gyroZ[rad/s]'][i][j]) + ', '
        for j in range(datanum):
            text += str(output_data['angleX[rad/s]'][i][j]) + ', '
        for j in range(datanum):
            text += str(output_data['angleY[rad/s]'][i][j]) + ', '
        for j in range(datanum):
            text += str(output_data['angleZ[rad/s]'][i][j]) + ', '
        for j in range(datanum):
            text += str(output_data['accX[m/s^2]'][i][j] / 9.8) + ', '
        for j in range(datanum):
            text += str(output_data['accY[m/s^2]'][i][j] / 9.8) + ', '
        for j in range(datanum):
            text += str(output_data['accZ[m/s^2]'][i][j] / 9.8) + ', '
        text += '\n'

    f.write(text)
    f.close

In [7]:
if __name__ == '__main__':
    directorys = ['data/191106_1152',
                  'data/191121_1625',
                  'data/191204_1956',
                  'data/191204_2012',
                  'data/191211_1649',
                  'data/200116_1104',
                  'data/200116_1121']

    for directory in directorys:
        df_acc = ld.loadAccData(directory+'/acc.csv')
        df_rtk = ld.loadRTKData(directory+'/rtk.csv')
        df_acc, df_rtk = ld.fitData(df_acc, df_rtk)

        print(directory + ' writing ...')
        output_data = makedata(df_rtk, df_acc)
        writedata(directory+'/ML.csv', output_data)
        output_data_angle = makedata_angle(df_rtk, df_acc)
        writedata_angle(directory+'/ML_angle.csv', output_data_angle)

data/191106_1152 writing ...
data/191121_1625 writing ...
data/191204_1956 writing ...
data/191204_2012 writing ...
data/191211_1649 writing ...
data/200116_1104 writing ...
data/200116_1121 writing ...
